# SQLite 데이터베이스 사용

SQLite는 메모리 또는 파일을 이용해 특별히 복잡한 설정 없이 SQL을 쓸 수 있는 데이터베이스입니다.

파이썬에선 `sqlite3` 모듈을 이용해 SQLite를 쓸 수 있습니다.

공식 문서: <https://docs.python.org/ko/3/library/sqlite3.html>

## 왜 SQLite를 쓰는가?

SQLite는 파일을 이용해 데이터를 기록하고 관리할 수 있습니다.

CSV 파일도 데이터를 기록하고 불러올 수 있지만, 일단 모두 불러온 다음에 Pandas 등을 통해 처리해야 합니다.

이렇게 되면

1. 데이터가 너무 크면 아예 불러오는 것 자체가 불가능합니다.
2. 효과적인 데이터 처리가 어렵습니다.

따라서 우리는 SQL을 사용해 이 문제를 해결할 수 있고, SQLite를 사용하면 가볍게 시작할 수 있습니다.

## sqlite 데이터베이스 파일 열기

데이터베이스는 연결/접속이란 개념을 사용합니다.
API 서버에 접속해서 데이터를 얻어온 것처럼, 우리는 데이터베이스에 접속해서 데이터를 CRUD할 겁니다.

여기서 CRUD는 다음과 같습니다:

1. Create
2. Read
3. Update
4. Delete

In [1]:
# sqlite3 모듈 사용

import sqlite3

sqlite3.version

'2.6.0'

In [2]:
# 파일로 기록되는 데이터베이스에 연결/접속

connection = sqlite3.connect('test.db')

## Cursor

실제로 데이터베이스를 사용할 땐 “커서(Cursor)”란 걸 활용합니다.

참고로, 우리가 컴퓨터로 글자를 입력할 때 현재 위치를 표시하는 박스 또는 세로 선도 커서라고 부르죠.

In [3]:
# 커서 얻기

cursor = connection.cursor()

## 테이블 만들기

SQL을 쓰려면 하나 이상의 테이블이 필요합니다.
테이블은 column과 row로 이뤄진 2차원 데이터라고 보시면 됩니다.

`CREATE TABLE` 명령으로 테이블을 만들 수 있고, 어떤 항목이 들어가는지 지정해야 합니다.
이걸 우리는 “스키마(Schema)”라고도 부릅니다.

SQL을 실행할 때는 `cursor`의 `execute`를 사용합니다.

In [4]:
sql = '''
CREATE TABLE users (
    name TEXT,
    age INTEGER,
    money REAL
)
'''

In [5]:
# 주의: 테이블이 이미 있을 땐 “table users already exists”라면서 실패하게 됩니다.

cursor.execute(sql)

## 데이터 추가 (Create)

데이터를 추가할 땐 `INSERT INTO`를 사용합니다.

In [6]:
cursor.execute('INSERT INTO users VALUES (?, ?, ?)', ['JOKER', 13, 3.14])

In [7]:
# 데이터를 실제로 파일에 기록하려면 “커밋”을 해야 합니다.

connection.commit()

## 데이터 얻기 (Read)

데이터를 얻을 땐 `SELECT`를 사용합니다.

In [8]:
# users 테이블의 모든 데이터를 얻습니다.

cursor.execute('SELECT * FROM users')

rows = cursor.fetchall()

print(rows)

[('JOKER', 13, 3.14)]


In [9]:
# 이름이 “JOKER”인 사용자 데이터를 하나만 얻습니다.

cursor.execute('SELECT * FROM users WHERE name=?', ['JOKER'])

row = cursor.fetchone()

print(row)

('JOKER', 13, 3.14)


## Row

데이터를 좀더 잘 다루기 위해 Row란 형태를 사용할 수 있습니다.

In [10]:
# Row를 사용하는 커서를 얻습니다.

connection.row_factory = sqlite3.Row

cursor = connection.cursor()

In [11]:
# 다른 커서를 사용하기 때문에 기존과 다르게 값이 바로 나오지 않습니다.

cursor.execute('SELECT * FROM users WHERE name=?', ['JOKER'])

row = cursor.fetchone()

print(row)

In [12]:
# 타입 확인

type(row)

sqlite3.Row

In [13]:
# Key 목록

row.keys()

['name', 'age', 'money']

In [14]:
# 이름 확인

row['name']

'JOKER'

In [15]:
# 모든 Value 출력
# dictionary와 다름에 주의!

for value in row:
    print(value)

JOKER
13
3.14


In [16]:
# 모든 Key-Value 출력

for key in row.keys():
    print(key, row[key])

name JOKER
age 13
money 3.14


In [17]:
# 더 많은 데이터 추가

for i in range(5):
    cursor.execute('INSERT INTO users VALUES (?, ?, ?)', [f'User {i}', 10 + i, i * 100])

connection.commit()

In [18]:
# 모든 데이터 출력

cursor.execute('SELECT * FROM users')

rows = cursor.fetchall()

for row in rows:
    for key in row.keys():
        print(key, row[key])
    print()

name JOKER
age 13
money 3.14

name User 0
age 10
money 0.0

name User 1
age 11
money 100.0

name User 2
age 12
money 200.0

name User 3
age 13
money 300.0

name User 4
age 14
money 400.0



## 모든 데이터 보기 함수

긴 코드가 보일 땐 함수로 만듭시다.

In [19]:
def print_users():
    cursor.execute('SELECT * FROM users')

    rows = cursor.fetchall()

    for row in rows:
        for key in row.keys():
            print(key, row[key])
        print()

In [20]:
print_users()

name JOKER
age 13
money 3.14

name User 0
age 10
money 0.0

name User 1
age 11
money 100.0

name User 2
age 12
money 200.0

name User 3
age 13
money 300.0

name User 4
age 14
money 400.0



## 데이터 수정 (Update)

UPDATE 명령을 사용해 데이터를 변경할 수 있습니다.

이때 WHERE를 쓰는 걸 잊지 마세요!

굉장히 많은 개발비화에 등장하는 게 “실수로 WHERE를 빼먹었다”죠.

In [21]:
# 이름이 “JOKER”인 사용자의 소지금을 13,579로 만듭니다.
# 파이썬에선 큰 숫자를 표현할 때 underscore(_)를 넣어서 사람이 보기 좋게 쓸 수 있습니다.

cursor.execute('UPDATE users SET money=? WHERE name=?', [13_579, 'JOKER'])

connection.commit()

In [22]:
print_users()

name JOKER
age 13
money 13579.0

name User 0
age 10
money 0.0

name User 1
age 11
money 100.0

name User 2
age 12
money 200.0

name User 3
age 13
money 300.0

name User 4
age 14
money 400.0



## 데이터 삭제 (Delete)

DELETE 명령을 사용해 데이터를 변경할 수 있습니다.

여기서도 WHERE를 쓰는 걸 잊지 마세요!

In [23]:
# 이름이 “JOKER”인 데이터를 삭제합니다.

cursor.execute('DELETE FROM users WHERE name=?', ['JOKER'])

connection.commit()

In [24]:
print_users()

name User 0
age 10
money 0.0

name User 1
age 11
money 100.0

name User 2
age 12
money 200.0

name User 3
age 13
money 300.0

name User 4
age 14
money 400.0



## DB 사용 종료

DB 사용을 마치면 연결/접속을 끊습니다.

In [25]:
connection.close()